In [ ]:
!pip install keras_applications
!pip install classification-models-3D
!pip install efficientnet-3D
!pip install segmentation-models-3D

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
imgarray=np.load('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/Origna_data_handling/t1/image_t1_list.npy')
maskarray=np.load('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/Origna_data_handling/masks3D/mask_list.npy')

In [ ]:
imgarray = imgarray[:,:,:,0:16]
maskarray = maskarray[:,:,:,0:16]

In [ ]:
x_train1,x_test,y_train1,y_test=train_test_split(imgarray, maskarray, test_size = 0.20, random_state = 101)
x_train1.shape, x_test.shape

((168, 128, 128, 16), (42, 128, 128, 16))

In [ ]:
def iou(y_true, y_pred, smooth = 100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def dice_coef(y_true, y_pred, smooth = 100):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    return K.mean(K.equal(y_true, K.round(y_pred)))

def iou(y_true, y_pred, smooth = 100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def dice_coef(y_true, y_pred, smooth = 100):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    return K.mean(K.equal(y_true, K.round(y_pred)))


In [ ]:
!pip install split-folders
import splitfolders  # or import split_folders

input_folder = '/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/Origna_data_handling/t2/'
output_folder = '/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/SplitData/'
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.8, .2), group_prefix=None) # default values

In [ ]:
import os
import numpy as np


def load_img(img_dir, img_list):
    images=[]
    for i, image_name in enumerate(img_list):    
        if (image_name.split('.')[1] == 'npy'):
            
            image = np.load(img_dir+image_name)
                      
            images.append(image)
    images = np.array(images)
    
    return(images)




def imageLoader(img_dir, img_list, mask_dir, mask_list, batch_size):

    L = len(img_list)

    #keras needs the generator infinite, so we will use while true  
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
                       
            X = load_img(img_dir, img_list[batch_start:limit])
            Y = load_img(mask_dir, mask_list[batch_start:limit])

            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size

In [ ]:
import os
import numpy as np
#from custom_datagen import imageLoader
#import tensorflow as tf
import keras
from matplotlib import pyplot as plt
import glob
import random

train_img_dir = "/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/SplitData/train/t2_np/"
train_mask_dir = "/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/SplitData/train/mask_np/"
train_img_list=os.listdir(train_img_dir)
train_mask_list = os.listdir(train_mask_dir)


val_img_dir = "/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/SplitData/val/t2_np/"
val_mask_dir = "/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/SplitData/val/mask_np/"
val_img_list=os.listdir(val_img_dir)
val_mask_list = os.listdir(val_mask_dir)

batch_size = 2

train_img_datagen = imageLoader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)

val_img_datagen = imageLoader(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list, batch_size)

#Verify generator.... In python 3 next() is renamed as __next__()
img, msk = train_img_datagen.__next__()

'''img_num = random.randint(0,img.shape[0]-1)
test_img=img[img_num]
test_mask=msk[img_num]
test_mask=np.argmax(test_mask, axis=3)

n_slice=random.randint(0, test_mask.shape[2])
plt.figure(figsize=(12, 8))

plt.subplot(121)
plt.imshow(test_img[:,:,n_slice], cmap='gray')
plt.title('Image t2')

plt.subplot(122)
plt.imshow(test_mask[:,:,n_slice])
plt.title('Mask')
plt.show()

#print((test_mask))'''

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Activation, Reshape
from keras.layers import BatchNormalization
import tensorflow as tf
from torch.nn import MaxUnpool3d
from keras.layers import Conv3D, MaxPooling3D, concatenate, UpSampling3D
from tensorflow.keras.optimizers import SGD


def segnet(
        input_shape,
        n_labels,
        kernel=3,
        pool_size=(2, 2, 2),
        output_mode="softmax"):
    # encoder
    inputs = Input(shape=input_shape)

    conv_1 = Conv3D(64, (kernel, kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    conv_2 = Conv3D(64, (kernel, kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation("relu")(conv_2)

    pool_1= MaxPooling3D(pool_size)(conv_2)

    conv_3 = Conv3D(128, (kernel, kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Conv3D(128, (kernel, kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Activation("relu")(conv_4)

    pool_2 = MaxPooling3D(pool_size)(conv_4)

    conv_5 = Conv3D(256, (kernel, kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Conv3D(256, (kernel, kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Activation("relu")(conv_6)
    conv_7 = Conv3D(256, (kernel, kernel, kernel), padding="same")(conv_6)
    conv_7 = BatchNormalization()(conv_7)
    conv_7 = Activation("relu")(conv_7)

    pool_3 = MaxPooling3D(pool_size)(conv_7)

    conv_8 = Conv3D(512, (kernel, kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Activation("relu")(conv_8)
    conv_9 = Conv3D(512, (kernel, kernel, kernel), padding="same")(conv_8)
    conv_9 = BatchNormalization()(conv_9)
    conv_9 = Activation("relu")(conv_9)
    conv_10 = Conv3D(512, (kernel, kernel, kernel), padding="same")(conv_9)
    conv_10 = BatchNormalization()(conv_10)
    conv_10 = Activation("relu")(conv_10)

    pool_4 = MaxPooling3D(pool_size)(conv_10)

    conv_11 = Conv3D(512, (kernel, kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Activation("relu")(conv_11)
    conv_12 = Conv3D(512, (kernel, kernel, kernel), padding="same")(conv_11)
    conv_12 = BatchNormalization()(conv_12)
    conv_12 = Activation("relu")(conv_12)
    conv_13 = Conv3D(512, (kernel, kernel, kernel), padding="same")(conv_12)
    conv_13 = BatchNormalization()(conv_13)
    conv_13 = Activation("relu")(conv_13)

    pool_5 = MaxPooling3D(pool_size)(conv_13)
    print("Build enceder done..")

    # decoder

    unpool_1 = UpSampling3D(pool_size)(pool_5)

    conv_14 = Conv3D(512, (kernel, kernel, kernel), padding="same")(unpool_1)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Activation("relu")(conv_14)
    conv_15 = Conv3D(512, (kernel, kernel, kernel), padding="same")(conv_14)
    conv_15 = BatchNormalization()(conv_15)
    conv_15 = Activation("relu")(conv_15)
    conv_16 = Conv3D(512, (kernel, kernel, kernel), padding="same")(conv_15)
    conv_16 = BatchNormalization()(conv_16)
    conv_16 = Activation("relu")(conv_16)

    unpool_2 = UpSampling3D(pool_size)(conv_16)

    conv_17 = Conv3D(512, (kernel, kernel, kernel), padding="same")(unpool_2)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_18 = Conv3D(512, (kernel, kernel, kernel), padding="same")(conv_17)
    conv_18 = BatchNormalization()(conv_18)
    conv_18 = Activation("relu")(conv_18)
    conv_19 = Conv3D(256, (kernel, kernel, kernel), padding="same")(conv_18)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_3 = UpSampling3D(pool_size)(conv_19)

    conv_20 = Conv3D(256, (kernel, kernel, kernel), padding="same")(unpool_3)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Conv3D(256, (kernel, kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Activation("relu")(conv_21)
    conv_22 = Conv3D(128, (kernel, kernel, kernel), padding="same")(conv_21)
    conv_22 = BatchNormalization()(conv_22)
    conv_22 = Activation("relu")(conv_22)

    unpool_4 = UpSampling3D(pool_size)(conv_22)

    conv_23 = Conv3D(128, (kernel, kernel, kernel), padding="same")(unpool_4)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Conv3D(64, (kernel, kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Activation("relu")(conv_24)

    unpool_5 = UpSampling3D(pool_size)(conv_24)

    conv_25 = Conv3D(64, (kernel, kernel, kernel), padding="same")(unpool_5)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu")(conv_25)

    conv_26 = Conv3D(n_labels, (1, 1, 1), padding="valid")(conv_25)
    conv_26 = BatchNormalization()(conv_26)
    '''conv_26 = Reshape(
            (input_shape[0]*input_shape[1], n_labels),
            input_shape=(input_shape[0], input_shape[1], n_labels))(conv_26)'''

    outputs = Activation(output_mode)(conv_26)
    print("Build decoder done..")
    #pred = Reshape((168, 128, 128, 16))(outputs)

    model = Model(inputs=inputs, outputs=outputs, name="SegNet")
    model.compile(optimizer= SGD(learning_rate=0.001, momentum=0.9, decay=0.0005, nesterov=False), loss= ["binary_crossentropy"]
                  , metrics=[iou, dice_coef, precision, recall, accuracy])
    model.summary()

    return model



model = segnet(input_shape=(168, 128, 128, 16), n_labels=4)


model.summary()
print(model.input_shape)
print(model.output_shape)

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import os
import numpy as np
from keras.utils import np_utils
from keras.applications import imagenet_utils


########################
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Activation, Reshape
from keras.layers import BatchNormalization
import tensorflow as tf
from torch.nn import MaxUnpool3d
from keras.layers import Conv3D, MaxPooling3D, concatenate, UpSampling3D


def SegNet(input_shape, classes):
    kernel=(3, 3, 3)
    pool_size=(2, 2, 2)
    output_mode="softmax"
    
    img_input = Input(shape=input_shape)
    x = img_input
    # Encoder
    x = Conv3D(64, kernel, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    pool_1 = MaxPooling3D(pool_size=pool_size)(x)
    
    x = Conv3D(128, kernel, padding="same")(pool_1)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    pool_2 = MaxPooling3D(pool_size=pool_size)(x)
    
    x = Conv3D(256, kernel, padding="same")(pool_2)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    pool_3 = MaxPooling3D(pool_size=pool_size)(x)
    
    x = Conv3D(512,kernel, padding="same")(pool_3)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    # Decoder
    x = Conv3D(512, kernel, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling3D(size=pool_size)(x)
    x = Conv3D(256, kernel, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling3D(size=pool_size)(x)
    x = Conv3D(128, kernel, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling3D(size=pool_size)(x)
    x = Conv3D(64, kernel, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv3D(classes, 1, 1, padding="valid")(x)
    #x = Reshape((input_shape[0]*input_shape[1]*input_shape[2], classes))(x)
    x = Activation("softmax")(x)
    model = Model(img_input, x)


    return model



model = SegNet(input_shape=(128,128,128,3), classes=4)

model.summary()
print(model.input_shape)
print(model.output_shape)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 128, 3) 0         
_________________________________________________________________
conv3d_52 (Conv3D)           (None, 128, 128, 128, 64) 5248      
_________________________________________________________________
batch_normalization_52 (Batc (None, 128, 128, 128, 64) 256       
_________________________________________________________________
activation_52 (Activation)   (None, 128, 128, 128, 64) 0         
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 64, 64, 64, 64)    0         
_________________________________________________________________
conv3d_53 (Conv3D)           (None, 64, 64, 64, 128)   221312    
_________________________________________________________________
batch_normalization_53 (Batc (None, 64, 64, 64, 128)   512   

In [ ]:
import tensorflow as tf

In [ ]:
!pip install keras_applications

In [ ]:
wt0, wt1, wt2, wt3 = 0.25,0.25,0.25,0.25
import segmentation_models_3D as sm
dice_loss = sm.losses.DiceLoss(class_weights=np.array([wt0, wt1, wt2, wt3])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

#metrics = ['accuracy', sm.metrics.IOUScore(threshold=0.5)]
metrics = ['accuracy']

LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
#######################################################################
#Fit the model 

steps_per_epoch = len(train_img_list)//batch_size
val_steps_per_epoch = len(val_img_list)//batch_size


model = SegNet((128,128,16, 1),4)
    # encoder
model.compile(optimizer= SGD(learning_rate=0.001, momentum=0.9, decay=0.0005, nesterov=False), loss= ["binary_crossentropy"]
                  , metrics=[iou, dice_coef, precision, recall, accuracy])
#model.compile(optimizer = optim, loss=total_loss, metrics=metrics)
#print(model.summary())

#print(model.input_shape)
#print(model.output_shape)

#x_train, y_train = imageLoader(train_img_dir, train_img_list, 
                                #train_mask_dir, train_mask_list, batch_size)
#x_val, y_val = imageLoader(val_img_dir, val_img_list, 
                                #val_mask_dir, val_mask_list, batch_size)


history=model.fit(train_img_datagen,
          steps_per_epoch=steps_per_epoch,
          epochs=30,
          verbose=1,
          validation_data=val_img_datagen,
          )

Epoch 1/30


ValueError: ignored

In [ ]:
x_train= imageLoader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)
x_val= imageLoader(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list, batch_size)

x_train, x_val

In [ ]:
import numpy as np
for i in range(20,25):
  my_mask =np.load('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/SplitData/val/t2_np/image_'+str(i)+'.npy')
  my_mask = my_mask[:,:,0:16].astype(np.float32)
  #my_mask = np.dtype(my_mask, np.uint8)
  np.save('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/SplitData/train/t2_np/image_'+str(i)+'.npy', my_mask)
  #maskarray = maskarray[:,:,0:16]
my_mask.shape, my_mask.dtype

((128, 128, 16), dtype('float32'))

In [ ]:
model = segnet(
        (168, 128, 128, 16),
        4)
hist = model.fit(x_train1, y_train1, epochs= 50, batch_size= 18, validation_data= (x_test, y_test), verbose=1)

In [ ]:
import numpy as np
import nibabel as nib
import glob
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


t2_list = sorted(glob.glob('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/DataSetBrats/Data/*/*t2.nii'))
mask_list = sorted(glob.glob('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/DataSetBrats/Data/*/*seg.nii'))

In [ ]:
for img in range(25):
    print("Now preparing image and masks number: ", img)
   

    temp_image_t2=nib.load(t2_list[img]).get_fdata()
    temp_image_t2=scaler.fit_transform(temp_image_t2.reshape(-1, temp_image_t2.shape[-1])).reshape(temp_image_t2.shape)
    temp_image_t2=temp_image_t2[56:184, 56:184, 13:141]
     
    temp_mask=nib.load(mask_list[img]).get_fdata()
    temp_mask=temp_mask.astype(np.uint8)
    temp_mask[temp_mask==4] = 3  #Reassign mask values 4 to 3
    temp_mask = temp_mask[56:184, 56:184, 13:141]
    #print(np.unique(temp_mask))
    
    
    #Crop to a size to be divisible by 64 so we can later extract 64x64x64 patches. 
    #cropping x, y, and z
    #temp_mask= to_categorical(temp_mask, num_classes=4)
    #np.save('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/Origna_data_handling/t1/images/image_'+str(img)+'.npy', temp_image_t1)
    #np.save('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/Origna_data_handling/t1ce/images/image_'+str(img)+'.npy', temp_image_t1ce)
    np.save('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/t2_mask/t2_np/image_'+str(img)+'.npy', temp_image_t2)
    #np.save('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/Origna_data_handling/flair/images/image_'+str(img)+'.npy', temp_image_flair)
    np.save('/content/drive/MyDrive/brain tumor segmentation/huzaifa data/example data/t2_mask/mask_np/mask_'+str(img)+'.npy', temp_mask) 
   
     